## Load a sample of crime data
Datasets used in this class are two from Seattle or San Francisco. Here I am first loading a small sample of the data to better understand how it is structured, and what interestign patterns might be present

[data source](https://github.com/djq/datasci_course_materials/blob/master/assignment6/crimeanalytics.md)



In [6]:
# first set my working directory
data = '/Users/djq/repos/jupyter-notebooks/communicating-data-science/week1/data'
setwd(data)

# load sample of data
seattle <- read.csv('seattle_incidents_summer_2014.csv')

# reduce this to first 100 rows
seattle_sample <- head(seattle, 100)

# what fields are available?
print(head(seattle_sample))

  RMS.CDW.ID General.Offense.Number Offense.Code Offense.Code.Extension
1     483839             2015218538         2202                      0
2     481252             2015213067         2610                      0
3     481375             2015210301         2316                      0
4     481690             2015209327         2599                      0
5     478198             2015207880         2399                      3
6     480485             2015904103         2308                      0
          Offense.Type Summary.Offense.Code Summarized.Offense.Description
1   BURGLARY-FORCE-RES                 2200                       BURGLARY
2 FRAUD-IDENTITY THEFT                 2600                          FRAUD
3           THEFT-MAIL                 2300                     MAIL THEFT
4          COUNTERFEIT                 2500                    COUNTERFEIT
5            THEFT-OTH                 2300                 OTHER PROPERTY
6       THEFT-BUILDING                 2300   